# TODO: Title

This notebook lists all the steps that you need to complete the complete this project. You will need to complete all the TODOs in this notebook as well as in the README and the two python scripts included with the starter code.


**TODO**: Give a helpful introduction to what this notebook is for. Remember that comments, explanations and good documentation make your project informative and professional.

**Note:** This notebook has a bunch of code and markdown cells with TODOs that you have to complete. These are meant to be helpful guidelines for you to finish your project while meeting the requirements in the project rubrics. Feel free to change the order of these the TODO's and use more than one TODO code cell to do all your tasks.

In [2]:
# TODO: Install any packages that you might need
# For instance, you will need the smdebug package
!pip install smdebug sagemaker boto3

  Using cached smdebug-1.0.34-py2.py3-none-any.whl.metadata (1.8 kB)
  Using cached protobuf-3.20.3-cp310-cp310-manylinux_2_12_x86_64.manylinux2010_x86_64.whl.metadata (679 bytes)
  Using cached pyinstrument-3.4.2-py2.py3-none-any.whl.metadata (22 kB)
  Using cached pyinstrument_cext-0.2.4-cp310-cp310-linux_x86_64.whl
Using cached smdebug-1.0.34-py2.py3-none-any.whl (280 kB)
Using cached pyinstrument-3.4.2-py2.py3-none-any.whl (83 kB)
Using cached protobuf-3.20.3-cp310-cp310-manylinux_2_12_x86_64.manylinux2010_x86_64.whl (1.1 MB)
  Attempting uninstall: protobuf
    Found existing installation: protobuf 4.25.3
    Uninstalling protobuf-4.25.3:
      Successfully uninstalled protobuf-4.25.3


In [3]:
# TODO: Import any packages that you might need
# For instance you will need Boto3 and Sagemaker
import sagemaker
import boto3
import os
import pandas as pd
import matplotlib.pyplot as plt
# Hyper parameter tuning
from sagemaker.tuner import (
    IntegerParameter,
    CategoricalParameter,
    ContinuousParameter,
    HyperparameterTuner,
)
from sagemaker.pytorch import PyTorch

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml


In [ ]:
#TODO: Fetch and upload the data to AWS S3
!wget https://s3-us-west-1.amazonaws.com/udacity-aind/dog-project/dogImages.zip
!unzip dogImages.zip

## Dataset
This project uses the dog breed classification dataset, which contains images of various dog breeds. The dataset is divided into training, validation, and testing sets. Each class in the dataset represents a different breed of dog.

The dataset can be downloaded from the following link and then uploaded to an S3 bucket for training:


In [4]:
BUCKET = 'sagemaker-us-east-1-912838639750'

# Upload metadata
sagemaker_session = sagemaker.Session()
PREFIX = 'Project/dog-breed-dataset'
# ---
DATASET_LOCAL_DIR = "dogImages"

In [27]:
dataset_dir_path = sagemaker_session.upload_data(
    path=DATASET_LOCAL_DIR,
    bucket=BUCKET,
    key_prefix=PREFIX
)

print(f"Dataset directory path on S3 : {dataset_dir_path}")

Dataset directory path on S3 : s3://sagemaker-us-east-1-912838639750/Project/dog-breed-dataset


In [5]:
dataset_dir_path = 's3://sagemaker-us-east-1-912838639750/Project/dog-breed-dataset'

## Hyperparameter Tuning
**TODO:** This is the part where you will finetune a pretrained model with hyperparameter tuning. Remember that you have to tune a minimum of two hyperparameters. However you are encouraged to tune more. You are also encouraged to explain why you chose to tune those particular hyperparameters and the ranges.

**Note:** You will need to use the `hpo.py` script to perform hyperparameter tuning.

In [14]:
#TODO: Declare your HP ranges, metrics etc.
# Setup hyperparameter search space
hyperparameter_ranges = {
    "lr": ContinuousParameter(0.001, 0.1),
    "batch-size": CategoricalParameter([32, 64, 128]),
    "epochs": CategoricalParameter([1, 2])
}

objective_metric_name = "average test loss"
objective_type = "Minimize"
metric_definitions = [{"Name": "average test loss", "Regex": "Testing Loss: ([0-9\\.]+)"}]

In [15]:
import os
# Create estimators for your hyperparameters
current_dir = os.getcwd()

# Initialize the model estimator
execution_role = sagemaker.get_execution_role()
model_estimator = PyTorch(
    entry_point=os.path.join(current_dir, "hpo.py"),
    role=execution_role,
    py_version='py36',
    framework_version="1.8",
    instance_count=1,
    instance_type="ml.c5.2xlarge"
)

hyperparameter_tuner = HyperparameterTuner(
    model_estimator,
    target_metric_name,
    hyperparameter_ranges,
    metric_patterns,
    max_jobs=3,
    max_parallel_jobs=3,
    objective_type=target_type,
)


In [16]:
# TODO: Fit your HP Tuner
valid_tuner_job_name = 'pytorch-tune-24070823'
hyperparameter_tuner.fit({'data': dataset_dir_path}, wait=True, job_name=valid_tuner_job_name)

INFO:sagemaker.image_uris:image_uri is not presented, retrieving image_uri based on instance_type, framework etc.
INFO:sagemaker:Creating hyperparameter tuning job with name: pytorch-tune-24070823


.......................................................................................................................................................................................................................................................!


In [17]:
# TODO: Get the best estimators and the best HPs

tuning_job_name = 'pytorch-tune-24070823'
hyperparameter_tuner = sagemaker.tuner.HyperparameterTuner.attach(tuning_job_name)

optimal_estimator = hyperparameter_tuner.best_estimator()

# Obtain the hyperparameters of the best model
optimal_hyperparameters = optimal_estimator.hyperparameters()
print(optimal_hyperparameters)


2024-07-08 14:21:37 Starting - Preparing the instances for training
2024-07-08 14:21:37 Downloading - Downloading the training image
2024-07-08 14:21:37 Training - Training image download completed. Training in progress.
2024-07-08 14:21:37 Uploading - Uploading generated training model
2024-07-08 14:21:37 Completed - Resource released due to keep alive period expiry
{'_tuning_objective_metric': '"mean_test_loss"', 'batch-size': '"64"', 'epochs': '"2"', 'lr': '0.006036452684336769', 'sagemaker_container_log_level': '20', 'sagemaker_estimator_class_name': '"PyTorch"', 'sagemaker_estimator_module': '"sagemaker.pytorch.estimator"', 'sagemaker_job_name': '"pytorch-tune-24070823"', 'sagemaker_program': '"hpo.py"', 'sagemaker_region': '"us-east-1"', 'sagemaker_submit_directory': '"s3://sagemaker-us-east-1-912838639750/pytorch-tune-24070823/source/sourcedir.tar.gz"'}


## Model Profiling and Debugging
TODO: Using the best hyperparameters, create and finetune a new model

**Note:** You will need to use the `train_model.py` script to perform model profiling and debugging.

In [33]:
# Creating the hyperparameters dictionary
optimal_hyperparameters = {
    'batch-size': '"64"',  # Example values with extra quotes
    'lr': '0.006036452684336769',
    'epochs': '"2"'
}

# Clean the values to ensure they are in the correct format
clean_hyperparameters = {
    "batch-size": int(optimal_hyperparameters['batch-size'].replace('"', '').strip()),
    "lr": float(optimal_hyperparameters['lr']),
    "epochs": int(optimal_hyperparameters['epochs'].replace('"', '').strip())
}

print(clean_hyperparameters)

{'batch-size': 64, 'lr': 0.006036452684336769, 'epochs': 2}


In [37]:
# TODO: Set up debugging and profiling rules and hooks

from sagemaker.debugger import Rule, ProfilerRule, rule_configs

rules = [ 
    Rule.sagemaker(rule_configs.loss_not_decreasing()),
    ProfilerRule.sagemaker(rule_configs.ProfilerReport())
]

In [38]:
# Set up the debugging and Profiler hooks
from sagemaker.debugger import DebuggerHookConfig, ProfilerConfig, FrameworkProfile, CollectionConfig

profiler_config = ProfilerConfig(
    system_monitor_interval_millis=500, framework_profile_params=FrameworkProfile()
)

collection_configs=[
    CollectionConfig(
        name="weights",
        parameters={
            "train.save_interval": "5",
            "eval.save_interval": "1"
        }
    ),
    CollectionConfig(
        name="gradients",
        parameters={
            "train.save_interval": "5",
            "eval.save_interval": "1"
        }
    ),
    CollectionConfig(
        name="biases",
        parameters={
            "train.save_interval": "5",
            "eval.save_interval": "1"
        }
    ),
    CollectionConfig(
        name="CrossEntopyLoss_output_0", 
        parameters={
            "include_regex": "CrossEntropyLoss_output_0",
            "train.save_interval": "1",
            "eval.save_interval": "1"
        }
    )
]

debugger_config = DebuggerHookConfig(
    hook_parameters={
        "train.save_interval": "5",
        "eval.save_interval": "1"
    },
    collection_configs=collection_configs
)

See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


In [49]:
# TODO: Create and fit an estimator

cwd = os.getcwd()

estimator_tuning = PyTorch(
    entry_point=os.path.join(cwd, "train_model.py"),
    base_job_name="Image-classification-Training",
    framework_version="1.8",
    py_version="py36",
    role=execution_role,
    instance_count=1,
    instance_type="ml.g4dn.xlarge",
    hyperparameters=clean_hyperparameters,
    rules=rules,
    profiler_config=profiler_config,
    debugger_hook_config=debugger_config,
)

In [50]:
# fitting the estimator
estimator_tuning.fit({'data': dataset_dir_path}, wait=True)

INFO:sagemaker.image_uris:image_uri is not presented, retrieving image_uri based on instance_type, framework etc.
INFO:sagemaker:Creating training-job with name: Image-classification-Training-2024-07-08-15-39-11-490


2024-07-08 15:39:12 Starting - Starting the training job...
2024-07-08 15:39:38 Starting - Preparing the instances for trainingLossNotDecreasing: InProgress
ProfilerReport: InProgress
...
2024-07-08 15:40:04 Downloading - Downloading input data......
2024-07-08 15:41:10 Downloading - Downloading the training image..................
2024-07-08 15:44:12 Training - Training image download completed. Training in progress...bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2024-07-08 15:44:22,763 sagemaker-training-toolkit INFO     Imported framework sagemaker_pytorch_container.training
2024-07-08 15:44:22,792 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
2024-07-08 15:44:22,796 sagemaker_pytorch_container.training INFO     Invoking user training script.
2024-07-08 15:44:23,062 sagemaker-training-toolkit INFO     Invoking user script
Training Env:
{
    "additional_framework_parameters": 

In [54]:
job_name = estimator_tuning.latest_training_job.name
client = estimator_tuning.sagemaker_session.sagemaker_client
description = client.describe_training_job(TrainingJobName=estimator_tuning.latest_training_job.name)

print(description)

{'TrainingJobName': 'Image-classification-Training-2024-07-08-15-39-11-490', 'TrainingJobArn': 'arn:aws:sagemaker:us-east-1:912838639750:training-job/Image-classification-Training-2024-07-08-15-39-11-490', 'ModelArtifacts': {'S3ModelArtifacts': 's3://sagemaker-us-east-1-912838639750/Image-classification-Training-2024-07-08-15-39-11-490/output/model.tar.gz'}, 'TrainingJobStatus': 'Completed', 'SecondaryStatus': 'Completed', 'HyperParameters': {'batch-size': '64', 'epochs': '2', 'lr': '0.006036452684336769', 'sagemaker_container_log_level': '20', 'sagemaker_job_name': '"Image-classification-Training-2024-07-08-15-39-11-490"', 'sagemaker_program': '"train_model.py"', 'sagemaker_region': '"us-east-1"', 'sagemaker_submit_directory': '"s3://sagemaker-us-east-1-912838639750/Image-classification-Training-2024-07-08-15-39-11-490/source/sourcedir.tar.gz"'}, 'AlgorithmSpecification': {'TrainingImage': '763104351884.dkr.ecr.us-east-1.amazonaws.com/pytorch-training:1.8-gpu-py36', 'TrainingInputMode

In [56]:
# TODO: Plot a debugging output.
import boto3

session = boto3.session.Session()
region = session.region_name

training_job_name = estimator_tuning.latest_training_job.name
print(f"Training jobname: {training_job_name}")
print(f"Region: {region}")


Training jobname: Image-classification-Training-2024-07-08-15-39-11-490
Region: us-east-1


In [58]:
from smdebug.trials import create_trial
from smdebug.core.modes import ModeKeys

trial = create_trial(estimator_tuning.latest_job_debugger_artifacts_path())

[2024-07-08 15:53:27.279 sagemaker-data-scienc-ml-t3-medium-ccb588b5efaf671be41927273f0c:20 INFO s3_trial.py:42] Loading trial debug-output at path s3://sagemaker-us-east-1-912838639750/Image-classification-Training-2024-07-08-15-39-11-490/debug-output
[2024-07-08 15:53:27.587 sagemaker-data-scienc-ml-t3-medium-ccb588b5efaf671be41927273f0c:20 WARNING s3handler.py:184] Encountered the exception An error occurred while reading from response stream: ('Connection broken: IncompleteRead(0 bytes read, 14114 more expected)', IncompleteRead(0 bytes read, 14114 more expected)) while reading s3://sagemaker-us-east-1-912838639750/Image-classification-Training-2024-07-08-15-39-11-490/debug-output/index/000000000/000000000032_worker_0.json . Will retry now


In [59]:
print(trial.tensor_names())

[2024-07-08 15:53:32.086 sagemaker-data-scienc-ml-t3-medium-ccb588b5efaf671be41927273f0c:20 INFO trial.py:197] Training has ended, will refresh one final time in 1 sec.
[2024-07-08 15:53:33.104 sagemaker-data-scienc-ml-t3-medium-ccb588b5efaf671be41927273f0c:20 INFO trial.py:210] Loaded all steps
['CrossEntropyLoss_output_0', 'ResNet_bn1.bias', 'ResNet_bn1.weight', 'ResNet_conv1.weight', 'ResNet_fc.bias', 'ResNet_fc.weight', 'ResNet_layer1.0.bn1.bias', 'ResNet_layer1.0.bn1.weight', 'ResNet_layer1.0.bn2.bias', 'ResNet_layer1.0.bn2.weight', 'ResNet_layer1.0.bn3.bias', 'ResNet_layer1.0.bn3.weight', 'ResNet_layer1.0.conv1.weight', 'ResNet_layer1.0.conv2.weight', 'ResNet_layer1.0.conv3.weight', 'ResNet_layer1.0.downsample.0.weight', 'ResNet_layer1.0.downsample.1.bias', 'ResNet_layer1.0.downsample.1.weight', 'ResNet_layer1.1.bn1.bias', 'ResNet_layer1.1.bn1.weight', 'ResNet_layer1.1.bn2.bias', 'ResNet_layer1.1.bn2.weight', 'ResNet_layer1.1.bn3.bias', 'ResNet_layer1.1.bn3.weight', 'ResNet_layer

In [60]:
print(len(trial.tensor('CrossEntropyLoss_output_0').steps(mode=ModeKeys.TRAIN)))
print(len(trial.tensor('CrossEntropyLoss_output_0').steps(mode=ModeKeys.EVAL)))

210
42


In [61]:
def get_data(trial, tname, mode):
    tensor = trial.tensor(tname)
    steps = tensor.steps(mode=mode)
    vals = []
    for s in steps:
        vals.append(tensor.value(s, mode=mode))
    return steps, vals

In [62]:
from mpl_toolkits.axes_grid1 import host_subplot


def plot_tensor(trial, tensor_name):

    steps_train, vals_train = get_data(trial, tensor_name, mode=ModeKeys.TRAIN)
    print("loaded TRAIN data")
    steps_eval, vals_eval = get_data(trial, tensor_name, mode=ModeKeys.EVAL)
    print("loaded EVAL data")

    fig = plt.figure(figsize=(10, 7))
    host = host_subplot(111)

    par = host.twiny()

    host.set_xlabel("Steps (TRAIN)")
    par.set_xlabel("Steps (EVAL)")
    host.set_ylabel(tensor_name)

    (p1,) = host.plot(steps_train, vals_train, label=tensor_name)
    print("completed TRAIN plot")
    (p2,) = par.plot(steps_eval, vals_eval, label="val_" + tensor_name)
    print("completed EVAL plot")
    leg = plt.legend()

    host.xaxis.get_label().set_color(p1.get_color())
    leg.texts[0].set_color(p1.get_color())

    par.xaxis.get_label().set_color(p2.get_color())
    leg.texts[1].set_color(p2.get_color())

    plt.ylabel(tensor_name)

    plt.show()

loaded TRAIN data
loaded EVAL data
completed TRAIN plot
completed EVAL plot


In [63]:
plot_tensor(trial, 'CrossEntropyLoss_output_0')

**TODO**: Is there some anomalous behaviour in your debugging output? If so, what is the error and how will you fix it?  
**TODO**: If not, suppose there was an error. What would that error look like and how would you have fixed it?

In [65]:
# TODO: Display the profiler output
rule_output_path = estimator_tuning.output_path + estimator_tuning.latest_training_job.job_name + "/rule-output"

In [66]:
! aws s3 ls {rule_output_path} --recursive

2024-07-08 15:49:06     390143 Image-classification-Training-2024-07-08-15-39-11-490/rule-output/ProfilerReport/profiler-output/profiler-report.html
2024-07-08 15:49:06     240253 Image-classification-Training-2024-07-08-15-39-11-490/rule-output/ProfilerReport/profiler-output/profiler-report.ipynb
2024-07-08 15:49:01        191 Image-classification-Training-2024-07-08-15-39-11-490/rule-output/ProfilerReport/profiler-output/profiler-reports/BatchSize.json
2024-07-08 15:49:01       8596 Image-classification-Training-2024-07-08-15-39-11-490/rule-output/ProfilerReport/profiler-output/profiler-reports/CPUBottleneck.json
2024-07-08 15:49:01       2380 Image-classification-Training-2024-07-08-15-39-11-490/rule-output/ProfilerReport/profiler-output/profiler-reports/Dataloader.json
2024-07-08 15:49:01        129 Image-classification-Training-2024-07-08-15-39-11-490/rule-output/ProfilerReport/profiler-output/profiler-reports/GPUMemoryIncrease.json
2024-07-08 15:49:01       1051 Image-classificat

In [67]:
! aws s3 cp {rule_output_path} ./ --recursive

download: s3://sagemaker-us-east-1-912838639750/Image-classification-Training-2024-07-08-15-39-11-490/rule-output/ProfilerReport/profiler-output/profiler-reports/CPUBottleneck.json to ProfilerReport/profiler-output/profiler-reports/CPUBottleneck.json
download: s3://sagemaker-us-east-1-912838639750/Image-classification-Training-2024-07-08-15-39-11-490/rule-output/ProfilerReport/profiler-output/profiler-reports/GPUMemoryIncrease.json to ProfilerReport/profiler-output/profiler-reports/GPUMemoryIncrease.json
download: s3://sagemaker-us-east-1-912838639750/Image-classification-Training-2024-07-08-15-39-11-490/rule-output/ProfilerReport/profiler-output/profiler-reports/BatchSize.json to ProfilerReport/profiler-output/profiler-reports/BatchSize.json
download: s3://sagemaker-us-east-1-912838639750/Image-classification-Training-2024-07-08-15-39-11-490/rule-output/ProfilerReport/profiler-output/profiler-reports/MaxInitializationTime.json to ProfilerReport/profiler-output/profiler-reports/MaxInit

In [69]:
import os

# get the autogenerated folder name of profiler report
profiler_report_name = [
    rule["RuleConfigurationName"]
    for rule in estimator_tuning.latest_training_job.rule_job_summary()
    if "Profiler" in rule["RuleConfigurationName"]
][0]

In [70]:
import IPython

IPython.display.HTML(filename=profiler_report_name + "/profiler-output/profiler-report.html")

## Model Deploying

In [31]:
from sagemaker.pytorch import PyTorchModel
model_data = estimator.output_path + estimator.latest_training_job.job_name + "/output/model.tar.gz"
print(f"Model: {model_data}")

Model: s3://sagemaker-us-east-1-912838639750/Image-classification-Project-debugging-2024-06-23-16-51-34-759/output/model.tar.gz


In [34]:
# TODO: Deploy your model to an endpoint
pytorch_model = PyTorchModel(
    model_data=model_data, 
    role=role, 
    entry_point='inference.py',
    py_version="py36",
    framework_version="1.8"
)

In [35]:
predictor = pytorch_model.deploy(initial_instance_count=1, instance_type="ml.t2.medium")

INFO:sagemaker:Repacking model artifact (s3://sagemaker-us-east-1-912838639750/Image-classification-Project-debugging-2024-06-23-16-51-34-759/output/model.tar.gz), script artifact (None), and dependencies ([]) into single tar.gz file located at s3://sagemaker-us-east-1-912838639750/pytorch-inference-2024-06-23-17-19-00-610/model.tar.gz. This may take some time depending on model size...
INFO:sagemaker:Creating model with name: pytorch-inference-2024-06-23-17-19-10-809
INFO:sagemaker:Creating endpoint-config with name pytorch-inference-2024-06-23-17-19-11-600
INFO:sagemaker:Creating endpoint with name pytorch-inference-2024-06-23-17-19-11-600


----------!

In [36]:
!pip install torchvision -q

In [37]:
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import torchvision.transforms as transforms

image_path = "./dogImages/test/011.Australian_cattle_dog/Australian_cattle_dog_00734.jpg"

In [38]:
# TODO: Run an prediction on the endpoint

image = Image.open(image_path)
transform = transforms.Compose([
            transforms.Resize(224),
            transforms.ToTensor(),
        ])
preprocessed_image = transform(image).unsqueeze(0)
preprocessed_image = preprocessed_image.to("cpu")
response = predictor.predict(preprocessed_image)

pred = np.argmax(response, 1) + 1

actual = int(image_path.split('.')[1].split('/')[-1])
print(f"Actual: {actual}, Prediction: {pred[0]}")

Actual: 11, Prediction: 11


In [39]:
# TODO: Remember to shutdown/delete your endpoint once your work is done
predictor.delete_endpoint()

INFO:sagemaker:Deleting endpoint configuration with name: pytorch-inference-2024-06-23-17-19-11-600
INFO:sagemaker:Deleting endpoint with name: pytorch-inference-2024-06-23-17-19-11-600
